In [3]:
import os
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="2"

In [4]:
import time
import torch
import numpy as np
import matplotlib.pyplot as plt

os.chdir("../") # So it finds the imports in upper directory, although it should.

from models import Cnet
from datasets import WSSBDatasetTest
from utils import od2rgb, rgb2od, random_ruifrok_matrix_variation, direct_deconvolution, peak_signal_noise_ratio

In [5]:
RUNNING_ON_DELFOS = False

torch.manual_seed(0)
plt.rcParams['font.size'] = 14
plt.rcParams['toolbar'] = 'None'

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [7]:
alsubaie_dataset_path = '/data/datasets/Alsubaie/Data/' if RUNNING_ON_DELFOS else '../Alsubaie/Data/'
organ_list = ['Colon']  # ['Lung', 'Breast', 'Colon']

In [ ]:
for organ in organ_list:
    dataset = WSSBDatasetTest(alsubaie_dataset_path, organ_list=organ, load_at_init=False)

    # Train the model and evaluate. 
    # Tiene la complicación de en que momento se para el entrenamiento de cada imagen y se suma la métrica para la media. Pensar